In [72]:

import transformers
import json
import sys
import os
import datasets
squad_dataset = datasets.load_dataset('squad')
from datasets import concatenate_datasets, load_dataset

covid_file = '../data/COVID-QA.json'
bio_file = '../bioASQ/bioASQ.json'

def make_and_save_full_dataset(train, valid, test, path):
    full_data = datasets.dataset_dict.DatasetDict({'train':train, 'validation':valid, 'test': test})
    full_data.save_to_disk(path)

def get_dataset(filename):
    return datasets.load_dataset('custom_squad.py', data_files= {'train':filename})['train']

Reusing dataset squad (/soe/jafidler/.cache/huggingface/datasets/squad/plain_text/1.0.0/4fffa6cf76083860f85fa83486ec3028e7e32c342c218ff2a620fc6b2868483a)


In [68]:
import run_qa

def run_gradual_ft(output_dir, checkpoint, covid_val):
    !python run_qa.py \
      --model_name_or_path {checkpoint} \
      --dataset_name ../data/full_squad_covidQA/ \
      --do_train \
      --do_eval \
      --do_predict \
      --per_device_train_batch_size 32\
      --per_device_eval_batch_size 32\
      --evaluation_strategy "no" \
      --save_strategy "no" \
      --logging_strategy "epoch" \
      --learning_rate 1e-5 \
      --num_train_epochs 1 \
      --max_seq_length 384 \
      --doc_stride 128 \
      --output_dir {output_dir} \
      --overwrite_output_dir

In [69]:
data_files = {}
data_files["train"] = covid_file

covid_qa = get_dataset(covid_file)
bio_qa = get_dataset(bio_file)

squad_qa = concatenate_datasets([squad_dataset['train'], squad_dataset['validation']])
covid_and_squad_dataset_path = "../data/full_squad_covidQA"

# squad_qa = datasets.Dataset.from_dict(squad_qa[:50])
# covid_qa = datasets.Dataset.from_dict(covid_qa[:20])
# bio_qa = datasets.Dataset.from_dict(bio_qa[:20])

Using custom data configuration default-1d18a620853d8414
Reusing dataset squad (/soe/jafidler/.cache/huggingface/datasets/squad/default-1d18a620853d8414/0.0.0/cb00e306c4924563ce3d1292a1ce1b86b2753dab6285ce43c87b39c5bda3ef4e)
Using custom data configuration default-d665e54172161a2c
Reusing dataset squad (/soe/jafidler/.cache/huggingface/datasets/squad/default-d665e54172161a2c/0.0.0/cb00e306c4924563ce3d1292a1ce1b86b2753dab6285ce43c87b39c5bda3ef4e)


In [ ]:
k_fold = 5
K = 6

for i in range(k_fold):
    covid_fold = covid_qa.shard(k_fold, i)

    covid_test = covid_fold.shard(2, 0)
    covid_val = covid_fold.shard(2, 1)
    covid_train = concatenate_datasets([covid_qa.shard(k_fold, j) for j in range(k_fold) if j != i])

    #make_and_save_full_dataset(covid_train, squad_qa, covid_val, covid_test, covid_and_squad_dataset_path)

    checkpoint = 'roberta-base'
    cur_dir = '../models/gradual_ft_baseline_lr1e-5/split_' + str(i)

    log_file = open(cur_dir + "/log_file.txt","w+")

    sys.stdout = log_file

    squad_qa.shuffle()
    bio_qa.shuffle()

    num_of_shards = int(K/2)
    squad_qa_shards = [squad_qa.shard(num_of_shards,i) for i in range(num_of_shards)]
    bio_qa_shards = [bio_qa.shard(num_of_shards,i) for i in range(num_of_shards)]

    squad_qa_cur = concatenate_datasets(squad_qa_shards)
    bio_qa_cur = concatenate_datasets(bio_qa_shards)
    
    squad_rows_to_remove = len(squad_qa_shards[0])
    bio_rows_to_remove = len(bio_qa_shards[0])
    L = squad_qa_shards + bio_qa_shards + [covid_train]
    print(f'Gradual finetuning with K={K} ({K//2} per SQuAD and bioASQ)')
    for n in range(K+1):
        output_dir = cur_dir + '/checkpoint_' + str(n)

       
        full_dataset = concatenate_datasets(L)
        make_and_save_full_dataset(full_dataset, covid_val, covid_test, covid_and_squad_dataset_path)

        print(f'n={n} =============')
        print('Total: ',full_dataset.num_rows)
        
        run_gradual_ft(output_dir, checkpoint, covid_val)
        print('=================\n')
       
        checkpoint = output_dir
    
        if n < K//2:
            print(f'Removing {squad_rows_to_remove} from SQuAD')
            _ = L.pop(0)
            
        elif n < K:
            print(f'Removing {bio_rows_to_remove} from bioASQ')
            _ = L.pop(0)

        else:
            assert full_dataset.num_rows == covid_train.num_rows
            full_dataset = covid_train
    
    log_file.close()
sys.stdout = sys.__stdout__